In [58]:
import os
import h5py
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from metrics import calculate_metric
from evaluate import*
import pandas as pd
import glob

In [44]:
len(HR0)
PRlength/18

39.44444444444444

In [59]:
def meanHR(PR,PRlength, HR_len):

    MeanPR = []
    for j in range(0,(PRlength//7)*7,PRlength//HR_len):
        PRmean = np.mean(PR[j:(j+(PRlength//HR_len))])
        MeanPR.append(PRmean)
    return np.array(MeanPR)

def scatter_plot_with_correlation_line(x, y,i,n,patnum, partname):
    # Create scatter plot
    
    xx = np.linspace(90,220,20)
    yy = np.linspace(90,220,20)
    plt.subplot(2,n,i+n).plot(xx, yy)
    plt.subplot(2,n,i+n).scatter(x, y)

#     # Add correlation line
#     axes = plt.gca()
#     m, b = np.polyfit(x, y, 1)
#     X_plot = np.linspace(axes.get_xlim()[0],axes.get_xlim()[1],100)
# #     plt.subplot(n//2,n-(n//2),i)
#     plt.subplot(2,n,i+n).plot(X_plot, m*X_plot + b, '-')
#     plt.xlim(90,220)
#     plt.ylim(90,220)
    plt.title(patnum+'_'+partname)
    
def bland_altman_params(data1, data2,i,n,patnum,partname, *args, **kwargs):
#     data = []
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0) 
    return mean,diff,md,sd
    
def bland_altman_plot(data1, data2,i,n,patnum,partname, *args, **kwargs):
#     data = []
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    
#     for i in range(diff.shape[0]):
#         if (md - 1.96*sd<diff[i] and diff[i] <md + 1.96*sd):
#             data.append([data1[i],data2[i]])
    plt.subplot(2,n,i).scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.axhline(md + sd, color='gray', linestyle='--')#1.96
    plt.axhline(md - sd, color='gray', linestyle='--')
    plt.title('{}_{}'.format(patnum,partname))
#     return np.array(data), md, sd

In [61]:
patnames[0]

'/home/nicky/nickynicu/Video-Based-Physiological-Measurement/deeplearning/ConventionalMethod/Whitebalance/phynetNegpea/outputppg_pearson/test/NICU/pat_06/part5'

In [47]:
len(HR)

17

In [102]:
%matplotlib inline
ppgPath = Path('/home/nicky/nickynicu/Video-Based-Physiological-Measurement/deeplearning/ConventionalMethod/Whitebalance/phynetNegpea/outputppg_pearson/test_360/NICU')
DATA_DIR='/home/nicky/nickynicu/Video-Based-Physiological-Measurement/deeplearning/ConventionalMethod/Whitebalance/conventionalResult/nicucrop'
GTpath = '/home/nicky/nickynicu/Video-Based-Physiological-Measurement/deeplearning/ConventionalMethod/Whitebalance/Datafolder/nicucrop'
patnames = glob.glob(os.path.join(ppgPath, "*pat*/part*"))
col = []
fcol = []
chromDict = {}
filtDict = {}
i = 1
plt.figure(figsize=(25,10))
for patpath in (patnames[:]):#+patnames[4:-3]):
#     print(patname)
    patnum, partname = patpath.split('_')[-1].split('/')
    patname = os.path.basename('/'.join(patpath.split('/')[:-1])) #os.path.basename(patname)
    vidpath = glob.glob(os.path.join(DATA_DIR,patname,'*/C0C',partname, "*CHROM"))[0]
    prediction = os.path.join(ppgPath,patname,partname, "{}.h5".format('ppgsignal'))
    label = os.path.join(ppgPath,patname,partname, "{}.h5".format('ppgGtsignal'))
    HRgtpath = glob.glob(os.path.join(GTpath,patname,'*/C0C',partname, 'HRgroundtruth.h5'))[0]
    if (Path(vidpath).exists()): 
        fvideo = h5py.File(vidpath, 'r')
        ppg = fvideo['ppg']
        HR = fvideo['HR']      
    if (Path(prediction).exists()):
        fpred = h5py.File(prediction, 'r')
        pred = fpred['ppgsignal']
    if (Path(label).exists()):
        flabel = h5py.File(label, 'r')
        label = flabel['ppgsignal']
    if (Path(HRgtpath).exists()):
        fHR = h5py.File(HRgtpath, 'r')
        HRGt = fHR['HRgt']
# # #     print(len(ppg), len(label),len(pred),len(HRgt))
    HR, HR0, MAE, RMSE, meanSNR = evluate(pred, label,window_size =100, fs = 25, signal = 'pulse')
    pearson = abs(np.corrcoef(HR, HR0)[1, 0])
    
    PRlength = (len(label)//25)*10

    HR0 = meanHR(HRGt,PRlength,len(HR))#[:-1]
    HRchrom = np.array(HR).squeeze() 
    
    mean,diff,md,sd = bland_altman_params(HR0, HR,i, 10,patnum, partname)

    HRVal = []
    for j in range(len(diff)):
        if diff[j]!=np.nan:
            if abs(diff[j])<(2*sd) :
                HRVal.append([HR[j], HR0[j]])
    if len(HRVal)==0:continue
    else:
        HRValid = np.array(HRVal).transpose(1,0)
        
    MAETrue = np.mean(np.abs(HR - HR0))
    RMSETrue = np.sqrt(np.mean(np.square(HR - HR0)))
    pearsonTrue = abs(np.corrcoef(HR, HR0)[1, 0])
        
    MAEchrom = np.mean(np.abs(HRValid[0] - HRValid[1]))
    RMSEchrom = np.sqrt(np.mean(np.square(HRValid[0] - HRValid[1])))
    pearsonchrom = abs(np.corrcoef(HRValid[0], HRValid[1])[1, 0])
        
    col.append('pat_{}_{}'.format(str(patnum), str(partname)))
    if 'pat_{}'.format(str(patnum)) in chromDict:
        chromDict['pat_{}_{}'.format(str(patnum),str(partname))].append([MAETrue,RMSETrue,pearsonTrue])
    else:
        chromDict['pat_{}_{}'.format(str(patnum), str(partname))] = [MAETrue,RMSETrue,pearsonTrue]
        
#     fcol.append('pat_{}_{}'.format(str(patnum), str(partname)))
    
    if 'pat_{}'.format(str(patnum)) in chromDict:
        filtDict['pat_{}_{}'.format(str(patnum),str(partname))].append([MAEchrom,RMSEchrom,pearsonchrom])
    else:
        filtDict['pat_{}_{}'.format(str(patnum), str(partname))] = [MAEchrom,RMSEchrom,pearsonchrom]

#     i+=1
    dfchrom = pd.DataFrame(chromDict,index=['MAE','RMSE','PEARSON'],columns=col)
    dffilt = pd.DataFrame(filtDict,index=['MAE','RMSE','PEARSON'],columns=col)
    
# #     pearson = abs(np.corrcoef(HR, HR0)[1, 0])
    
#     #comparison with gt HR
#     gtPath = sorted(Path(basepath).glob('PR/S009_s0{}*.xlsx'.format(patnum)))[-1]
#     gtFile = pd.read_excel(gtPath,index_col='Frame', usecols = "A,B")
# #     for j in range(len(gtFile)):
# #         if gtFile['PR'][j+1]==0: print('PR value not there')
    
#     PRlength = (len(ppg)//25)*10
# #     if patnum == '06':
# #         HRgt = meanHR_06(gtPath,PRlength)
# #     else:
#     HRgt = meanHR(GtHR,len(GtHR), len(HR))
#     print(len(HR),len(HRgt))
    
# # # #     HRgt = meanGtHR(HR,PRlength)
    
# #     print('w.r.t chrom:',patname, HR, HR0, MAE, RMSE, meanSNR,pearson)
#     col.append('{}'.format(str(patname)))
#     if '{}_{}'.format(str(patname), str(partname)) in chromDict:
#         print('in dict')
#         chromDict['{}_{}'.format(str(patname), str(partname))].append([MAE, RMSE,pearson])
#     else:
#         chromDict['{}_{}'.format(str(patname), str(partname))] = [MAE, RMSE,pearson]
#     HRGt = HRgt[-len(HR):].squeeze()
#     MAEgt = np.mean(np.abs(HR - HRGt))
#     RMSEgt = np.sqrt(np.mean(np.square(HR - HRGt)))
#     pearsongt = abs(np.corrcoef(HR, HRGt)[1, 0])
#     if '{}_{}'.format(str(patname), str(partname)) in gtDict:
#         gtDict['{}_{}'.format(str(patname), str(partname))].append([MAEgt, RMSEgt,pearsongt])
#     else:
#         gtDict['{}_{}'.format(str(patname), str(partname))] = [MAEgt, RMSEgt,pearsongt]
# #     scatter_plot_with_correlation_line(HRGt, HR,i,len(patnames)+1,patnum, partname)
#     bland_altman_plot(HRGt, HR,i,len((patnames[:7]+patnames[8:]))+1,patnum, partname)
#     scatter_plot_with_correlation_line(HRGt, HR,i,len((patnames[:7]+patnames[8:]))+1,patnum, partname)
    i += 1
# # # #     print(patname,HR,HRGt)
# # #     print('w.r.t GT:', patname,HR,HRGt, MAEgt, RMSEgt, pearsongt)
#     break
display(dfchrom)
display(dffilt)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.893056,18.159722,16.468056,13.600000,10.494444,26.697222,3.312500,6.020833,33.180556,8.140278,33.933333,40.401389,10.991667,10.869444,12.254167,25.659722
RMSE,21.671064,22.901340,19.576225,19.970243,13.966137,54.023265,5.125098,7.843944,37.672972,11.268357,37.455841,45.142434,13.109245,13.623346,15.266706,30.642612
PEARSON,0.473806,0.311110,0.027544,0.178635,0.038021,0.278632,0.234502,0.352450,0.256111,0.229098,0.189565,0.027860,0.179158,0.043596,0.301757,0.016496


,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,14.760417,12.235714,16.468056,7.628333,8.764706,15.836765,2.467647,5.066176,25.442857,5.859375,21.757500,23.000000,10.123529,9.802941,8.788333,18.501786
RMSE,15.523972,14.871746,19.576225,9.581277,10.624455,21.522877,3.071633,6.002117,27.708437,7.404158,22.652156,25.778692,11.956487,12.126102,10.225069,21.585781
PEARSON,0.003608,0.063046,0.027544,0.525297,0.025095,0.520648,0.195767,0.351670,0.095911,0.501311,0.815313,0.367305,0.131644,0.151655,0.158750,0.319083


<Figure size 1800x720 with 0 Axes>

In [101]:
display(dfchrom)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.893056,18.159722,16.468056,13.600000,10.494444,26.697222,3.312500,6.020833,33.180556,8.140278,33.933333,40.401389,10.991667,10.869444,12.254167,25.659722
RMSE,21.671064,22.901340,19.576225,19.970243,13.966137,54.023265,5.125098,7.843944,37.672972,11.268357,37.455841,45.142434,13.109245,13.623346,15.266706,30.642612
PEARSON,0.473806,0.311110,0.027544,0.178635,0.038021,0.278632,0.234502,0.352450,0.256111,0.229098,0.189565,0.027860,0.179158,0.043596,0.301757,0.016496


In [97]:
display(dfchrom)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.645833,18.576389,18.343056,10.350000,11.350000,26.922222,3.443056,6.256944,34.847222,8.348611,38.516667,42.693056,13.491667,9.827778,12.670833,29.979167
RMSE,21.138916,23.132375,22.503690,17.003621,13.524702,53.751479,5.172641,7.996507,39.371266,11.336098,43.064419,46.523916,15.832068,12.551632,15.821907,34.825621
PEARSON,0.432147,0.307477,0.116445,0.089601,0.158683,0.324668,0.192566,0.302124,0.404589,0.247699,0.071744,0.015216,0.322050,0.054264,0.325208,0.161542


In [93]:
display(dfchrom)
display(dffilt)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.136298,16.348278,15.180416,8.442970,13.114778,18.723099,4.256098,4.473458,31.104017,8.557030,35.663199,40.577834,12.334648,9.355452,13.062052,26.470947
RMSE,21.056063,21.354923,17.895169,10.738587,15.797521,36.608806,6.933128,5.682010,36.250275,11.391475,39.794607,45.670042,13.959676,12.493266,16.371962,32.125321
PEARSON,0.289304,0.318237,0.090299,0.279359,0.132351,0.248814,0.398188,0.386882,0.036933,0.243020,0.157559,0.111432,0.151437,0.162256,0.011018,0.018128


,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,13.686807,14.016387,15.180416,6.726829,11.930640,11.402439,2.389024,4.065549,20.196138,7.268293,19.659299,23.262873,11.495427,6.973171,10.776423,19.719077
RMSE,14.456884,17.452289,17.895169,8.370669,14.174453,16.445483,3.165853,5.171128,21.390318,9.201662,20.801633,26.070885,12.867456,8.812622,13.490781,22.927629
PEARSON,0.077249,0.263451,0.090299,0.285207,0.131177,0.351759,0.278014,0.357048,0.823361,0.360235,0.733840,0.244614,0.098494,0.106028,0.350355,0.340551


In [84]:
display(dfchrom)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.893056,18.159722,16.468056,13.600000,10.494444,26.697222,3.312500,6.020833,33.180556,8.140278,33.933333,40.401389,10.991667,10.869444,12.254167,25.659722
RMSE,21.671064,22.901340,19.576225,19.970243,13.966137,54.023265,5.125098,7.843944,37.672972,11.268357,37.455841,45.142434,13.109245,13.623346,15.266706,30.642612
PEARSON,0.473806,0.311110,0.027544,0.178635,0.038021,0.278632,0.234502,0.352450,0.256111,0.229098,0.189565,0.027860,0.179158,0.043596,0.301757,0.016496


In [81]:
display(dfchrom)
display(dffilt)
###

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.645833,18.576389,18.343056,10.350000,11.350000,26.922222,3.443056,6.256944,34.847222,8.348611,38.516667,42.693056,13.491667,9.827778,12.670833,29.979167
RMSE,21.138916,23.132375,22.503690,17.003621,13.524702,53.751479,5.172641,7.996507,39.371266,11.336098,43.064419,46.523916,15.832068,12.551632,15.821907,34.825621
PEARSON,0.432147,0.307477,0.116445,0.089601,0.158683,0.324668,0.192566,0.302124,0.404589,0.247699,0.071744,0.015216,0.322050,0.054264,0.325208,0.161542


,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,17.030000,12.771429,16.160294,5.870312,11.350000,16.075000,2.605882,5.316176,25.548077,6.093750,20.925000,22.653571,12.079412,7.679688,9.038333,19.529167
RMSE,17.866160,15.324586,18.850475,7.527608,13.524702,20.789890,3.154886,6.211620,27.983081,7.519589,21.780858,24.587673,13.515849,9.248418,10.480436,22.235131
PEARSON,0.284696,0.060386,0.027881,0.358897,0.158683,0.561281,0.297616,0.281126,0.064794,0.513708,0.736291,0.331849,0.276610,0.199468,0.119590,0.010421


In [77]:
display(dfchrom)
display(dffilt)

,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,19.136298,16.348278,15.180416,8.442970,13.114778,18.723099,4.256098,4.473458,31.104017,8.557030,35.663199,40.577834,12.334648,9.355452,13.062052,26.470947
RMSE,21.056063,21.354923,17.895169,10.738587,15.797521,36.608806,6.933128,5.682010,36.250275,11.391475,39.794607,45.670042,13.959676,12.493266,16.371962,32.125321
PEARSON,0.289304,0.318237,0.090299,0.279359,0.132351,0.248814,0.398188,0.386882,0.036933,0.243020,0.157559,0.111432,0.151437,0.162256,0.011018,0.018128


,pat_06_part5,pat_06_part7,pat_06_part6,pat_06_part3,pat_02_part2,pat_07_part4,pat_20_part7,pat_20_part6,pat_11_part7,pat_16_part7,pat_03_part5,pat_03_part4,pat_19_part7,pat_19_part6,pat_19_part4,pat_09_part4
MAE,13.686807,14.016387,15.180416,6.726829,11.930640,11.402439,2.389024,4.065549,20.196138,7.268293,19.659299,23.262873,11.495427,6.973171,10.776423,19.719077
RMSE,14.456884,17.452289,17.895169,8.370669,14.174453,16.445483,3.165853,5.171128,21.390318,9.201662,20.801633,26.070885,12.867456,8.812622,13.490781,22.927629
PEARSON,0.077249,0.263451,0.090299,0.285207,0.131177,0.351759,0.278014,0.357048,0.823361,0.360235,0.733840,0.244614,0.098494,0.106028,0.350355,0.340551


In [103]:
metricsFilePath = '/home/nicky/nickynicu/Video-Based-Physiological-Measurement/deeplearning/ConventionalMethod/Whitebalance/phynetNegpea/metrics/intra1_360'
os.makedirs(metricsFilePath, exist_ok=True)

In [104]:
chromMetrics = metricsFilePath +'/chromMetrics.csv'
dfchrom.to_csv(chromMetrics,index = False)
chromMetricsfiltered = metricsFilePath +'/chromMetricsfiltered.csv'
dffilt.to_csv(chromMetricsfiltered,index = False)

In [ ]:
df